In [1]:
import pandas as pd

# Cargar el dataset
file_path = "../../datasets/clasified_transactions_to_use.csv" # Cambia por la ruta de tu archivo CSV
df = pd.read_csv(file_path)

# Crear transacciones a partir de las columnas clasificadas
columns_to_analyze = ['categoria', 'popularidad', 'duracion_segundosclassified', 
                      'promedio_tiempo_reproducidoclassified', 'impresionesclassified', 
                      'tasa_de_clicksclassified', 'vistasclassified']

# Combinar características de cada fila en una lista de transacciones
transactions = df[columns_to_analyze].apply(lambda row: list(row), axis=1).tolist()

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Codificar las transacciones en formato binario
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

# Calcular itemsets frecuentes
frequent_itemsets = apriori(df_encoded, min_support=0.1, use_colnames=True)

# Generar reglas de asociación
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Mostrar resultados
print(rules)

# Filtrar reglas con alta confianza
filtered_rules <- rules[rules["confidence"] > 0.8]
print(filtered_rules)



ModuleNotFoundError: No module named 'mlxtend'